## This notebook simulates NMDAR hypoactivation together with changes to IPSC decay times as requested by Reviewer 1
NMDAR hypoactivation is modelled by changing the parameter b at inhibitory cells (either basket or chandelier cells) to more negative values ([-0.0125, -0.015, -0.0175, -0.02]). This mimics the decreased excitability of the cells when the NMDAergic excitatory component is decreased. Additionally, IPSC decay times (via the tau parameter of the synapse model) is prolonged for either basket or chandelier cells.

In [2]:
%matplotlib inline 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab

import sys
sys.path.append("/home/cmetzner/Chandelier-Basket-Model/")

### Import necessary model and analysis modules

In [3]:
from Scripts.analysis import calcPowerSpectrum
from Model.simple_model_chandelier_class import simpleModelChandelier
from Scripts.plot_utility import plot_MEG_summary_overlay_4in1, plot_PSD_summary_overlay_4in1

### Define model parameters

ChC changes excitability (parameter b) + ChC changes inhibitory decay time

In [4]:
# Parameters
n_ex            = 80              # number of excitatory cells


eta             = 5.0             # 
tau_R           = 0.1             # 'synaptic rise time'
tau_ex          = 2.0             # exc. 'synaptic decay time'
tau_bask        = 8.0             # inh. 'synaptic decay time'  for basket cell
tau_chand_ctrl  = 8.0             # inh. 'synaptic decay time'  for chandelier cell
tau_chand_schiz = 28.0             # inh. 'synaptic decay time'  for chandelier cell

g_ee            = 0.00375         # E-E weight
g_eb            = 0.00625         # default=0.025 E-B weight
g_ec            = 0.00625         # default=0.025 E-C weight
g_be            = 0.00375         # default=0.015 B-E weight
g_ce_ctrl       = 0.00375         # default=0.015 C-E weight
g_ce_schiz      = 0.00375         # default=0.015 C-E weight
g_bb            = 0.005           # default=0.02 B-B weight
g_cc            = 0.0             # no C-C inhibition
g_bc            = 0.005           # default=0.02 B-C weight
g_de            = 0.3             # default=0.3 Drive-E weight
g_db            = 0.08            # default=0.08 Drive-B weight
g_dc            = 0.08            # default=0.08 Drive-C weight


s               = 2**13
sim_time        = 500             # simulation stime (in ms) 
dt              = float(sim_time)/float(s)
time            = np.linspace(0,sim_time,int(sim_time/dt)+1)

b_ex            = -0.01           # default=-0.01 applied current for exc. cells
b_bask          = -0.01           # default=-0.01 applied current for basket cells
b_chand         = -0.01
b_chand_schizs  = [-0.0125, -0.015, -0.0175, -0.02]  # default=-0.01 applied current for chandelier cells
background_rate = 33.3
A               = 0.6

seeds           = np.load('../Data/Avg-firing-rate-noise-only-seeds.npy')
directory       = ''

frequencies     = [20.0,30.0,40.0]
chc_numbers     = [20,10,4,2]
chc_percentages = ['50','25','10','05']

### Simulate models

In [5]:
data = []
for k,b_chand_schiz in enumerate(b_chand_schizs):
    for j,n_chand in enumerate(chc_numbers):
        print 'Chandelier percentage:',chc_percentages[j]
        n_bask = (n_ex/2)-n_chand # number of bask cells
        chc_per = chc_percentages[j]
        data_n =[]
        for f in frequencies:
            drive_frequency = f

            meg_ctrl_avg    = np.zeros((8193,))
            meg_schiz_avg   = np.zeros((8193,))
            for i,seed in enumerate(seeds):
                print i
                filename_ctrl   = '../Data/Figure_NMDAR_ChC_IPSC_ChC/Single-Trial-Data/Data-'+str(int(f))+'Hz/Ctrl_ChPer'+chc_per+'-'+str(int(f))+'Hz'+'_'+str(i)
                filename_schiz  = '../Data/Figure_NMDAR_ChC_IPSC_ChC/Single-Trial-Data/Data-'+str(int(f))+'Hz/Schiz_ChPer'+chc_per+'-'+str(int(f))+'Hz'+'_'+str(i)

                # Control model
                model_ctrl = simpleModelChandelier(n_ex,n_bask,n_chand,eta,tau_R,tau_ex,tau_bask,tau_chand_ctrl,g_ee,g_eb,g_ec,g_be,g_ce_ctrl,g_bb,g_cc,g_bc,g_de,g_db,g_dc,dt,b_ex,b_bask,b_chand,drive_frequency,background_rate,A,seed,filename_ctrl,directory)
                meg_ctrl,ex_ctrl,bask_ctrl,chand_ctrl = model_ctrl.run(sim_time,1,1,1,1)
                meg_ctrl_avg += meg_ctrl   


                # Schizophrenia-like model
                model_schiz = simpleModelChandelier(n_ex,n_bask,n_chand,eta,tau_R,tau_ex,tau_bask,tau_chand_schiz,g_ee,g_eb,g_ec,g_be,g_ce_schiz,g_bb,g_cc,g_bc,g_de,g_db,g_dc,dt,b_ex,b_bask,b_chand_schiz,drive_frequency,background_rate,A,seed,filename_schiz,directory)
                meg_schiz,ex_schiz,bask_schiz,chand_schiz = model_schiz.run(sim_time,1,1,1,1)
                meg_schiz_avg += meg_schiz   


            meg_ctrl_avg = meg_ctrl_avg /len(seeds)
            np.save('../Data/Figure_NMDAR_ChC_IPSC_ChC/Ctrl-ChPer'+chc_per+'-MEG-'+str(int(f))+'Hz',meg_ctrl_avg)

            meg_schiz_avg = meg_schiz_avg /len(seeds)
            np.save('../Data/Figure_NMDAR_ChC_IPSC_ChC/Schiz-ChPer'+chc_per+'-MEG-'+str(int(f))+'Hz',meg_schiz_avg)

            data_n.append((meg_ctrl_avg,meg_schiz_avg)) 

        data.append(data_n)
    
np.save('../Data/figNMDARChCIPSCChC.npy',data)

Chandelier percentage: 50
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
Chandelier percentage: 25
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
Chandelier percentage: 10
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
Chandelier percentage: 05
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
Chandelier percentage: 50
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
Chandelier percentage: 25
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17


### Define model parameters

ChC changes excitability (parameter b) + BC changes inhibitory decay time

In [6]:
# Parameters
n_ex            = 80              # number of excitatory cells


eta             = 5.0             # 
tau_R           = 0.1             # 'synaptic rise time'
tau_ex          = 2.0             # exc. 'synaptic decay time'
tau_bask        = 8.0             # inh. 'synaptic decay time'  for basket cell
tau_chand       = 8.0             # inh. 'synaptic decay time'  for chandelier cell
tau_bask_schiz  = 28.0             # inh. 'synaptic decay time'  for chandelier cell

g_ee            = 0.00375         # E-E weight
g_eb            = 0.00625         # default=0.025 E-B weight
g_ec            = 0.00625         # default=0.025 E-C weight
g_be            = 0.00375         # default=0.015 B-E weight
g_ce_ctrl       = 0.00375         # default=0.015 C-E weight
g_ce_schiz      = 0.00375         # default=0.015 C-E weight
g_bb            = 0.005           # default=0.02 B-B weight
g_cc            = 0.0             # no C-C inhibition
g_bc            = 0.005           # default=0.02 B-C weight
g_de            = 0.3             # default=0.3 Drive-E weight
g_db            = 0.08            # default=0.08 Drive-B weight
g_dc            = 0.08            # default=0.08 Drive-C weight


s               = 2**13
sim_time        = 500             # simulation stime (in ms) 
dt              = float(sim_time)/float(s)
time            = np.linspace(0,sim_time,int(sim_time/dt)+1)

b_ex            = -0.01           # default=-0.01 applied current for exc. cells
b_bask          = -0.01           # default=-0.01 applied current for basket cells
b_chand         = -0.01
b_chand_schizs  = [-0.0125, -0.015, -0.0175, -0.02]  # default=-0.01 applied current for chandelier cells
background_rate = 33.3
A               = 0.6

seeds           = np.load('../Data/Avg-firing-rate-noise-only-seeds.npy')
directory       = ''

frequencies     = [20.0,30.0,40.0]
chc_numbers     = [20,10,4,2]
chc_percentages = ['50','25','10','05']

### Simulate models


In [ ]:
data = []
for k,b_chand_schiz in enumerate(b_chand_schizs):
    for j,n_chand in enumerate(chc_numbers):
        print 'Chandelier percentage:',chc_percentages[j]
        n_bask = (n_ex/2)-n_chand # number of bask cells
        chc_per = chc_percentages[j]
        data_n =[]
        for f in frequencies:
            drive_frequency = f

            meg_ctrl_avg    = np.zeros((8193,))
            meg_schiz_avg   = np.zeros((8193,))
            for i,seed in enumerate(seeds):
                print i
                filename_ctrl   = '../Data/Figure_NMDAR_ChC_IPSC_BC/Single-Trial-Data/Data-'+str(int(f))+'Hz/Ctrl_ChPer'+chc_per+'-'+str(int(f))+'Hz'+'_'+str(i)
                filename_schiz  = '../Data/Figure_NMDAR_ChC_IPSC_BC/Single-Trial-Data/Data-'+str(int(f))+'Hz/Schiz_ChPer'+chc_per+'-'+str(int(f))+'Hz'+'_'+str(i)

                # Control model
                model_ctrl = simpleModelChandelier(n_ex,n_bask,n_chand,eta,tau_R,tau_ex,tau_bask,tau_chand,g_ee,g_eb,g_ec,g_be,g_ce_ctrl,g_bb,g_cc,g_bc,g_de,g_db,g_dc,dt,b_ex,b_bask,b_chand,drive_frequency,background_rate,A,seed,filename_ctrl,directory)
                meg_ctrl,ex_ctrl,bask_ctrl,chand_ctrl = model_ctrl.run(sim_time,1,1,1,1)
                meg_ctrl_avg += meg_ctrl   


                # Schizophrenia-like model
                model_schiz = simpleModelChandelier(n_ex,n_bask,n_chand,eta,tau_R,tau_ex,tau_bask_schiz,tau_chand,g_ee,g_eb,g_ec,g_be,g_ce_schiz,g_bb,g_cc,g_bc,g_de,g_db,g_dc,dt,b_ex,b_bask,b_chand_schiz,drive_frequency,background_rate,A,seed,filename_schiz,directory)
                meg_schiz,ex_schiz,bask_schiz,chand_schiz = model_schiz.run(sim_time,1,1,1,1)
                meg_schiz_avg += meg_schiz   


            meg_ctrl_avg = meg_ctrl_avg /len(seeds)
            np.save('../Data/Figure_NMDAR_ChC_IPSC_BC/Ctrl-ChPer'+chc_per+'-MEG-'+str(int(f))+'Hz',meg_ctrl_avg)

            meg_schiz_avg = meg_schiz_avg /len(seeds)
            np.save('../Data/Figure_NMDAR_ChC_IPSC_BC/Schiz-ChPer'+chc_per+'-MEG-'+str(int(f))+'Hz',meg_schiz_avg)

            data_n.append((meg_ctrl_avg,meg_schiz_avg)) 

        data.append(data_n)
    
np.save('../Data/figNMDARChCIPSCBC.npy',data)

Chandelier percentage: 50
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
Chandelier percentage: 25
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
Chandelier percentage: 10
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
Chandelier percentage: 05
0
1
2
3
4
5
6
7
8


### Define model parameters

BC changes excitability (parameter b) + ChC changes inhibitory decay time

In [ ]:
# Parameters
n_ex            = 80              # number of excitatory cells


eta             = 5.0             # 
tau_R           = 0.1             # 'synaptic rise time'
tau_ex          = 2.0             # exc. 'synaptic decay time'
tau_bask        = 8.0             # inh. 'synaptic decay time'  for basket cell
tau_chand_ctrl  = 8.0             # inh. 'synaptic decay time'  for chandelier cell
tau_chand_schiz = 28.0            # inh. 'synaptic decay time'  for chandelier cell

g_ee            = 0.00375         # E-E weight
g_eb            = 0.00625         # default=0.025 E-B weight
g_ec            = 0.00625         # default=0.025 E-C weight
g_be            = 0.00375         # default=0.015 B-E weight
g_ce_ctrl       = 0.00375         # default=0.015 C-E weight
g_ce_schiz      = 0.00375         # default=0.015 C-E weight
g_bb            = 0.005           # default=0.02 B-B weight
g_cc            = 0.0             # no C-C inhibition
g_bc            = 0.005           # default=0.02 B-C weight
g_de            = 0.3             # default=0.3 Drive-E weight
g_db            = 0.08            # default=0.08 Drive-B weight
g_dc            = 0.08            # default=0.08 Drive-C weight


s               = 2**13
sim_time        = 500             # simulation stime (in ms) 
dt              = float(sim_time)/float(s)
time            = np.linspace(0,sim_time,int(sim_time/dt)+1)

b_ex            = -0.01           # default=-0.01 applied current for exc. cells
b_bask          = -0.01           # default=-0.01 applied current for basket cells
b_bask_schizs  = [-0.0125, -0.015, -0.0175, -0.02]  # default=-0.01 applied current for chandelier cells
b_chand         = -0.01
background_rate = 33.3
A               = 0.6

seeds           = np.load('../Data/Avg-firing-rate-noise-only-seeds.npy')
directory       = ''

frequencies     = [20.0,30.0,40.0]
chc_numbers     = [20,10,4,2]
chc_percentages = ['50','25','10','05']

### Simulate models

In [ ]:
data = []
for k,b_bask_schiz in enumerate(b_bask_schizs):
    for j,n_chand in enumerate(chc_numbers):
        print 'Chandelier percentage:',chc_percentages[j]
        n_bask = (n_ex/2)-n_chand # number of bask cells
        chc_per = chc_percentages[j]
        data_n =[]
        for f in frequencies:
            drive_frequency = f

            meg_ctrl_avg    = np.zeros((8193,))
            meg_schiz_avg   = np.zeros((8193,))
            for i,seed in enumerate(seeds):
                print i
                filename_ctrl   = '../Data/Figure_NMDAR_BC_IPSC_ChC/Single-Trial-Data/Data-'+str(int(f))+'Hz/Ctrl_ChPer'+chc_per+'-'+str(int(f))+'Hz'+'_'+str(i)
                filename_schiz  = '../Data/Figure_NMDAR_BC_IPSC_ChC/Single-Trial-Data/Data-'+str(int(f))+'Hz/Schiz_ChPer'+chc_per+'-'+str(int(f))+'Hz'+'_'+str(i)

                # Control model
                model_ctrl = simpleModelChandelier(n_ex,n_bask,n_chand,eta,tau_R,tau_ex,tau_bask,tau_chand_ctrl,g_ee,g_eb,g_ec,g_be,g_ce_ctrl,g_bb,g_cc,g_bc,g_de,g_db,g_dc,dt,b_ex,b_bask,b_chand,drive_frequency,background_rate,A,seed,filename_ctrl,directory)
                meg_ctrl,ex_ctrl,bask_ctrl,chand_ctrl = model_ctrl.run(sim_time,1,1,1,1)
                meg_ctrl_avg += meg_ctrl   


                # Schizophrenia-like model
                model_schiz = simpleModelChandelier(n_ex,n_bask,n_chand,eta,tau_R,tau_ex,tau_bask,tau_chand_schiz,g_ee,g_eb,g_ec,g_be,g_ce_schiz,g_bb,g_cc,g_bc,g_de,g_db,g_dc,dt,b_ex,b_bask_schiz,b_chand,drive_frequency,background_rate,A,seed,filename_schiz,directory)
                meg_schiz,ex_schiz,bask_schiz,chand_schiz = model_schiz.run(sim_time,1,1,1,1)
                meg_schiz_avg += meg_schiz   


            meg_ctrl_avg = meg_ctrl_avg /len(seeds)
            np.save('../Data/Figure_NMDAR_BC_IPSC_ChC/Ctrl-ChPer'+chc_per+'-MEG-'+str(int(f))+'Hz',meg_ctrl_avg)

            meg_schiz_avg = meg_schiz_avg /len(seeds)
            np.save('../Data/Figure_NMDAR_BC_IPSC_ChC/Schiz-ChPer'+chc_per+'-MEG-'+str(int(f))+'Hz',meg_schiz_avg)

            data_n.append((meg_ctrl_avg,meg_schiz_avg)) 

        data.append(data_n)
    
np.save('../Data/figNMDARBCIPSCChC.npy',data)

### Define model parameters

BC changes excitability (parameter b) + BC changes inhibitory decay time

In [ ]:
# Parameters
n_ex            = 80              # number of excitatory cells


eta             = 5.0             # 
tau_R           = 0.1             # 'synaptic rise time'
tau_ex          = 2.0             # exc. 'synaptic decay time'
tau_bask        = 8.0             # inh. 'synaptic decay time'  for basket cell
tau_chand  = 8.0             # inh. 'synaptic decay time'  for chandelier cell
tau_bask_schiz = 28.0            # inh. 'synaptic decay time'  for chandelier cell

g_ee            = 0.00375         # E-E weight
g_eb            = 0.00625         # default=0.025 E-B weight
g_ec            = 0.00625         # default=0.025 E-C weight
g_be            = 0.00375         # default=0.015 B-E weight
g_ce_ctrl       = 0.00375         # default=0.015 C-E weight
g_ce_schiz      = 0.00375         # default=0.015 C-E weight
g_bb            = 0.005           # default=0.02 B-B weight
g_cc            = 0.0             # no C-C inhibition
g_bc            = 0.005           # default=0.02 B-C weight
g_de            = 0.3             # default=0.3 Drive-E weight
g_db            = 0.08            # default=0.08 Drive-B weight
g_dc            = 0.08            # default=0.08 Drive-C weight


s               = 2**13
sim_time        = 500             # simulation stime (in ms) 
dt              = float(sim_time)/float(s)
time            = np.linspace(0,sim_time,int(sim_time/dt)+1)

b_ex            = -0.01           # default=-0.01 applied current for exc. cells
b_bask          = -0.01           # default=-0.01 applied current for basket cells
b_bask_schizs  = [-0.0125, -0.015, -0.0175, -0.02]  # default=-0.01 applied current for chandelier cells
b_chand         = -0.01
background_rate = 33.3
A               = 0.6

seeds           = np.load('../Data/Avg-firing-rate-noise-only-seeds.npy')
directory       = ''

frequencies     = [20.0,30.0,40.0]
chc_numbers     = [20,10,4,2]
chc_percentages = ['50','25','10','05']

### Simulate models

In [ ]:
data = []
for k,b_bask_schiz in enumerate(b_bask_schizs):
    for j,n_chand in enumerate(chc_numbers):
        print 'Chandelier percentage:',chc_percentages[j]
        n_bask = (n_ex/2)-n_chand # number of bask cells
        chc_per = chc_percentages[j]
        data_n =[]
        for f in frequencies:
            drive_frequency = f

            meg_ctrl_avg    = np.zeros((8193,))
            meg_schiz_avg   = np.zeros((8193,))
            for i,seed in enumerate(seeds):
                print i
                filename_ctrl   = '../Data/Figure_NMDAR_BC_IPSC_BC/Single-Trial-Data/Data-'+str(int(f))+'Hz/Ctrl_ChPer'+chc_per+'-'+str(int(f))+'Hz'+'_'+str(i)
                filename_schiz  = '../Data/Figure_NMDAR_BC_IPSC_BC/Single-Trial-Data/Data-'+str(int(f))+'Hz/Schiz_ChPer'+chc_per+'-'+str(int(f))+'Hz'+'_'+str(i)

                # Control model
                model_ctrl = simpleModelChandelier(n_ex,n_bask,n_chand,eta,tau_R,tau_ex,tau_bask,tau_chand,g_ee,g_eb,g_ec,g_be,g_ce_ctrl,g_bb,g_cc,g_bc,g_de,g_db,g_dc,dt,b_ex,b_bask,b_chand,drive_frequency,background_rate,A,seed,filename_ctrl,directory)
                meg_ctrl,ex_ctrl,bask_ctrl,chand_ctrl = model_ctrl.run(sim_time,1,1,1,1)
                meg_ctrl_avg += meg_ctrl   


                # Schizophrenia-like model
                model_schiz = simpleModelChandelier(n_ex,n_bask,n_chand,eta,tau_R,tau_ex,tau_bask_schiz,tau_chand,g_ee,g_eb,g_ec,g_be,g_ce_schiz,g_bb,g_cc,g_bc,g_de,g_db,g_dc,dt,b_ex,b_bask_schiz,b_chand,drive_frequency,background_rate,A,seed,filename_schiz,directory)
                meg_schiz,ex_schiz,bask_schiz,chand_schiz = model_schiz.run(sim_time,1,1,1,1)
                meg_schiz_avg += meg_schiz   


            meg_ctrl_avg = meg_ctrl_avg /len(seeds)
            np.save('../Data/Figure_NMDAR_BC_IPSC_BC/Ctrl-ChPer'+chc_per+'-MEG-'+str(int(f))+'Hz',meg_ctrl_avg)

            meg_schiz_avg = meg_schiz_avg /len(seeds)
            np.save('../Data/Figure_NMDAR_BC_IPSC_BC/Schiz-ChPer'+chc_per+'-MEG-'+str(int(f))+'Hz',meg_schiz_avg)

            data_n.append((meg_ctrl_avg,meg_schiz_avg)) 

        data.append(data_n)
    
np.save('../Data/figNMDARBCIPSCBC.npy',data)